In [1]:
import sys
import torch
from torch.utils.data import Subset
import torchvision.models as tvmodels
from torchvision import datasets, transforms

sys.path.insert(0, "../src")
import exlib
from exlib.modules import *

In [2]:
IMAGENET_VAL_DIR = "/data/imagenet/val"
imagenet = datasets.ImageFolder(
    IMAGENET_VAL_DIR,
    transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
)

torch.manual_seed(123)
imagenet = Subset(imagenet, indices=torch.randperm(len(imagenet)))

In [3]:
MODEL_PATH = "/home/antonxue/foo/mus/saved_models/" \
             + "vit16_durt_psz28_ft__0.5000_0.2500__epoch5.pt"
state_dict = torch.load(MODEL_PATH)
vit16 = tvmodels.vit_b_16()
vit16.load_state_dict(state_dict)
mus_vit16 = VisionMuS(vit16, q=64, lambd=0.50, patch_size=28, \
                      image_shape=torch.Size([3,224,224]))
mus_vit16.eval();

In [4]:
x0, y0 = imagenet[0]
x1, y1 = imagenet[1]
x2, y2 = imagenet[2]
X = torch.stack([x0, x1, x2])

In [5]:
with torch.no_grad():
    y = vit16(X)

y.argmax(dim=1), [y0, y1, y2]

(tensor([247, 508, 165]), [247, 620, 165])